# Car Price Prediction::

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

#1.Load Data

In [ ]:
import numpy as np
import pandas as pd
data =  pd.read_csv('CarPrice_Assignment.csv')
data

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3217,ohc,six,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


#2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

In [ ]:
data.isnull().sum() #no null values.

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [ ]:
data.info() # from this command we'll be able to see all the info. since we can't feed string values(object) into our model so we'll encode them.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data['fueltype'] = labelencoder.fit_transform(data['fueltype'])
data['aspiration'] = labelencoder.fit_transform(data['aspiration'])
data['carbody'] = labelencoder.fit_transform(data['carbody'])
data['drivewheel'] = labelencoder.fit_transform(data['drivewheel'])
data['enginelocation'] = labelencoder.fit_transform(data['enginelocation'])
data['enginetype'] = labelencoder.fit_transform(data['enginetype'])
data['fuelsystem'] = labelencoder.fit_transform(data['fuelsystem'])
#data['cylindernumber'] = labelencoder.fit_transform(data['cylindernumber'])
#data['doornumber'] = labelencoder.fit_transform(data['doornumber'])
data['doornumber'] = data['doornumber'].map({'two':2,'four':4}) #we can do it like this also.
data['cylindernumber'] = data['cylindernumber'].map({'two':2,'three':3,'four':4,'five':5,'six':6,'eight':8,'twelve':12}) 


In [ ]:
#we are going to check correlation between independent and dependent variables such that the x variables with high correlation with y will be taken further for prediction of y.
import seaborn as sns
import matplotlib.pyplot as plt
corrmatrics = data.corr()
highcorr_features =  corrmatrics.index
plt.figure(figsize=(20,20))
plot = sns.heatmap(data[highcorr_features].corr(),annot=True,cmap='RdYlGn')

In [ ]:
#From above heatmap we can see the correlation is the highest for followeing features:
data1 = data[['price','horsepower','boreratio','fuelsystem','enginesize','curbweight','carwidth','carlength','wheelbase','drivewheel']]
data1.shape

(205, 10)

In [ ]:
#Normalizing the data
from sklearn import preprocessing
d = preprocessing.normalize(data1)
names = ['price','horsepower','boreratio','fuelsystem','enginesize','curbweight','carwidth','carlength','wheelbase','drivewheel']
data1_scaled = pd.DataFrame(d, columns=names)
data1_scaled.head()

,price,horsepower,boreratio,fuelsystem,enginesize,curbweight,carwidth,carlength,wheelbase,drivewheel
0,0.982457,0.008081,0.000253,0.000364,0.009464,0.185498,0.004667,0.012289,0.006450,0.000146
1,0.988162,0.006648,0.000208,0.000299,0.007786,0.152596,0.003839,0.010109,0.005306,0.000120
2,0.985520,0.009198,0.000160,0.000299,0.009079,0.168614,0.003912,0.010226,0.005644,0.000119
3,0.986089,0.007210,0.000225,0.000353,0.007705,0.165196,0.004680,0.012483,0.007055,0.000071
4,0.987035,0.006505,0.000180,0.000283,0.007693,0.159736,0.003756,0.009989,0.005622,0.000000


#3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

In [ ]:
x = data1_scaled.loc[:,data1_scaled.columns != 'price']
y = data1_scaled.loc[:,data1_scaled.columns == 'price']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.30,random_state=42)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(143, 9) (143, 1)
(62, 9) (62, 1)


In [ ]:
x_train1,x_validation,y_train1,y_validation = train_test_split(x,y,test_size = 0.2,random_state = 42)

print(x_train1.shape,y_train1.shape)
print(x_validation.shape,y_validation.shape)

(164, 9) (164, 1)
(41, 9) (41, 1)


#4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
network = models.Sequential()
network.add(layers.Dense(10, activation='relu', input_shape=(9,)))
network.add(layers.Dense(8, activation='relu'))
network.add(layers.Dense(6, activation='relu'))
network.add(layers.Dense(1))

#5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)

In [ ]:
network.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

#6.Train the Model with Epochs (100) and validate it

In [ ]:
network.fit(x_train, y_train, epochs=100, batch_size=128)

Epoch 1/100
2/2 [==============================] - 1s 6ms/step - loss: 0.9130 - mae: 0.9553
Epoch 2/100
2/2 [==============================] - 0s 4ms/step - loss: 0.8727 - mae: 0.9339
Epoch 3/100
2/2 [==============================] - 0s 8ms/step - loss: 0.8462 - mae: 0.9196
Epoch 4/100
2/2 [==============================] - 0s 5ms/step - loss: 0.8247 - mae: 0.9078
Epoch 5/100
2/2 [==============================] - 0s 4ms/step - loss: 0.8066 - mae: 0.8978
Epoch 6/100
2/2 [==============================] - 0s 5ms/step - loss: 0.7878 - mae: 0.8872
Epoch 7/100
2/2 [==============================] - 0s 5ms/step - loss: 0.7718 - mae: 0.8782
Epoch 8/100
2/2 [==============================] - 0s 4ms/step - loss: 0.7549 - mae: 0.8685
Epoch 9/100
2/2 [==============================] - 0s 5ms/step - loss: 0.7399 - mae: 0.8598
Epoch 10/100
2/2 [==============================] - 0s 4ms/step - loss: 0.7245 - mae: 0.8508
Epoch 11/100
2/2 [==============================] - 0s 4ms/step - loss: 0.7093 

In [ ]:
test_mse_score, test_mae_score = network.evaluate(x_test, y_test)

2/2 [==============================] - 0s 5ms/step - loss: 0.0040 - mae: 0.0514


#7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need.

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
network1 = models.Sequential()
network1.add(layers.Dense(4, activation='relu', input_shape=(9,))) #decreasing the number of neurons.
network1.add(layers.Dense(3, activation='relu'))
network1.add(layers.Dense(2, activation='relu'))
network1.add(layers.Dense(1))
#network1.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
network1.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
network1.fit(x_train, y_train, epochs=50, batch_size=128)       #decreasing epochs.
test_mse_score, test_mae_score = network1.evaluate(x_test, y_test)

Epoch 1/50
2/2 [==============================] - 1s 9ms/step - loss: 0.9463 - mae: 0.9727
Epoch 2/50
2/2 [==============================] - 0s 5ms/step - loss: 0.9366 - mae: 0.9677
Epoch 3/50
2/2 [==============================] - 0s 6ms/step - loss: 0.9294 - mae: 0.9640
Epoch 4/50
2/2 [==============================] - 0s 3ms/step - loss: 0.9233 - mae: 0.9608
Epoch 5/50
2/2 [==============================] - 0s 11ms/step - loss: 0.9181 - mae: 0.9580
Epoch 6/50
2/2 [==============================] - 0s 5ms/step - loss: 0.9136 - mae: 0.9557
Epoch 7/50
2/2 [==============================] - 0s 6ms/step - loss: 0.9087 - mae: 0.9532
Epoch 8/50
2/2 [==============================] - 0s 7ms/step - loss: 0.9050 - mae: 0.9512
Epoch 9/50
2/2 [==============================] - 0s 5ms/step - loss: 0.8999 - mae: 0.9485
Epoch 10/50
2/2 [==============================] - 0s 4ms/step - loss: 0.8969 - mae: 0.9469
Epoch 11/50
2/2 [==============================] - 0s 4ms/step - loss: 0.8924 - mae: 0.9

#8.Evaluation Step

In [ ]:
test_loss, test_acc = network.evaluate(x_test, y_test) 
print('test_acc:', test_acc)

test_loss, test_acc = network1.evaluate(x_test, y_test) 
print('test_acc:', test_acc)

2/2 [==============================] - 0s 6ms/step - loss: 0.0040 - mae: 0.0514
test_acc: 0.05143294855952263
2/2 [==============================] - 0s 6ms/step - loss: 0.7466 - mae: 0.8640
test_acc: 0.8639904260635376


#9.Prediction

In [ ]:
network.predict(x_test)  #prediction is positive since the values are greater than 0.5

array([[0.87316436],
       [0.9239668 ],
       [0.9806058 ],
       [0.9664839 ],
       [0.8937696 ],
       [0.9966082 ],
       [1.0178065 ],
       [0.97401655],
       [0.97925854],
       [1.0174999 ],
       [0.9481023 ],
       [1.0005347 ],
       [0.9064319 ],
       [0.9808021 ],
       [0.8651565 ],
       [1.03305   ],
       [1.0230261 ],
       [0.96476036],
       [1.0180275 ],
       [0.9757545 ],
       [1.0114291 ],
       [0.96773034],
       [0.98422503],
       [1.0015213 ],
       [1.023497  ],
       [0.8548484 ],
       [0.99542654],
       [0.96620053],
       [0.9928457 ],
       [0.9659952 ],
       [0.89797086],
       [1.037059  ],
       [1.016057  ],
       [0.90128183],
       [1.0085665 ],
       [0.88746524],
       [0.9654635 ],
       [0.9617509 ],
       [0.96242446],
       [0.94765514],
       [1.0286155 ],
       [0.90723073],
       [0.9445832 ],
       [1.0441408 ],
       [0.99932766],
       [0.9881231 ],
       [1.037059  ],
       [1.024

In [ ]:
network1.predict(x_test)

array([[0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.10824907],
       [0.108